# MDLH

In [ ]:
!pip install tensorflow==1.15

In [ ]:
import matplotlib.pyplot as plt
from tick.hawkes import ModelHawkesExpKernLogLik
from tick.hawkes import SimuHawkesExpKernels, SimuHawkesMulti, HawkesExpKern
from tick.hawkes import HawkesADM4, HawkesCumulantMatching
import numpy as np
import pandas as pd
from scipy.stats import gamma
from scipy.stats import expon
from scipy.stats import uniform
from scipy.stats import bernoulli
import pickle
import itertools
import tensorflow
%precision 3

In [ ]:
def generate_data_uniform(end_time, decays, gamma):
    p = len(gamma)
    adjacency =  gamma * np.random.uniform(low=uni_adj_lb, high=uni_adj_ub,size=(p,p))
    #print(np.max(np.abs(np.linalg.eigvals(adjacency))))
    #print(np.count_nonzero(adjacency) - p)
    baseline = np.random.uniform(low=uni_bas_lb, high=uni_bas_ub,size=p)
    true = SimuHawkesExpKernels(
        adjacency=adjacency, decays=decays, baseline=baseline,
        end_time=end_time*2, verbose=False)
    obs = SimuHawkesMulti(true, n_simulations=1)
    obs.simulate()
    data = obs.timestamps[0].copy()
    for i in range(p):
        data[i] = data[i][data[i] > end_time] - end_time
    #print(len(data[0]))
    return data,baseline,adjacency

In [ ]:
def generate_data_uniform_multi(end_time, decays, gamma, n_sim):
    p = len(gamma)
    adjacency =  gamma * np.random.uniform(low=uni_adj_lb, high=uni_adj_ub,size=(p,p))
    #print(np.max(np.abs(np.linalg.eigvals(adjacency))))
    #print(np.count_nonzero(adjacency) - p)
    baseline = np.random.uniform(low=uni_bas_lb, high=uni_bas_ub,size=p)
    true = SimuHawkesExpKernels(
        adjacency=adjacency, decays=decays, baseline=baseline,
        end_time=end_time*2, verbose=False)
    obs = SimuHawkesMulti(true, n_simulations=n_sim)
    obs.simulate()
    data = obs.timestamps.copy()
    for j in range(n_sim):
        for i in range(p):
            data[j][i] = data[j][i][data[j][i] > end_time] - end_time
    #print(len(data[0][0]))
    return data,baseline,adjacency

In [ ]:
def estimate(data,s):
    p = len(data)
    r = np.zeros(p)
    for i in range(p):
        r[p-i-1] = s%2
        s = s//2
    r = r==1
    dat = [data[i] for i in range(p) if (r[i])]
    ls = HawkesExpKern(decays, penalty='none',gofit = 'least-squares',step = 10, max_iter=10000000, tol = 1e-5)
    ls.fit(dat)
    #print(ls.adjacency)
    #print(ls.baseline)
    mle = HawkesExpKern(decays, penalty='none', gofit = 'likelihood', step = 1, max_iter=10000000, tol = 1e-5, solver = 'gd')
    mle.fit(events=dat,start = ls.coeffs+1e-5)
    #print(mle.adjacency)
    return mle

In [ ]:
def generate_gamma(p):
    gamma = bernoulli(bern_param).rvs((p,p))
    for i in range(p):
        gamma[i][i] = 1 
    gamma = gamma == 1
    return gamma

In [ ]:
def generate_gamma_sparse(p,deg):
    gamma = np.diag(np.ones(p))
    for i in range(p):
        u = np.random.choice(range(0,deg+1))
        r = np.zeros(p-1)
        r[:u] = 1
        np.random.shuffle(r)
        o = 0
        for j in range(p-1):
            if (j==i):
                o = 1
            else:
                gamma[i][j+o] = r[j]
    gamma = gamma == 1
    return gamma

In [ ]:
def est_eval(data):
    p = len(data)
    adj = [0]
    bas = [0]
    for s in range(1,2**p):
        #print(s)
        mle = estimate(data,s)
        adj.append(mle.adjacency)
        bas.append(mle.baseline)
    return adj,bas

In [ ]:
def est_eval_sparse(data,deg):
    p = len(data)
    adj = dict()
    bas = dict()
    tot = []
    for u in range(1,deg+2):
        for w in itertools.combinations(range(p),u):
            tot.append(w)
    for w in tot:
        s = 0
        for c in w:
            s += 2**c
        mle = estimate(data,s)
        adj[s] = mle.adjacency
        bas[s] = mle.baseline
    return adj,bas

In [ ]:
def theta_hat_eval(adj,bas,gamma):
    p = len(gamma)
    a = np.zeros((p,p))
    b = np.zeros(p)
    for i in range(p):
        s = 0
        for x in gamma[i]:
            s = s*2 + x
        u = 0
        for j in range(i):
            if (gamma[i][j]):
                u+=1
        b[i] = bas[s][u]
        w = 0
        for j in range(p):
            if (gamma[i][j] == 1):
                a[i][j] = adj[s][u][w]
                w += 1
    coeffs = np.concatenate([b,a.flatten()]) #p+p^2 entries
    return coeffs

In [ ]:
def q(theta_hat,theta,data,gamma_hat,gamma):
    L = ModelHawkesExpKernLogLik(decay=decays,n_threads=0)
    L.fit(data);
    pr_hat = np.exp(-L.loss(coeffs=theta_hat))
    pr = np.exp(-L.loss(coeffs=theta))
    return (pr_hat/pr)

In [ ]:
def COMP(gamma,prep):
    prep_data_list,prep_adj_list,prep_bas_list,prep_gamma_list,prep_theta_list = prep
    x = []
    N = len(prep_adj_list)
    for i in range(N):
        theta_hat = theta_hat_eval(adj=prep_adj_list[i],bas = prep_bas_list[i],gamma=gamma)
        u = q(theta_hat=theta_hat,theta=prep_theta_list[i],data = prep_data_list[i],gamma_hat=gamma,gamma=prep_gamma_list[i])
        x.append(u)
    #print(np.mean(x))
    #print(np.std(x))
    return np.log(np.mean(x))

In [ ]:
def generate_sample(p,T):
    gamma_true= generate_gamma(p)
    data_true,baseline_true,adjacency_true = generate_data_uniform(end_time=T,decays=decays,gamma = gamma_true)
    return data_true,gamma_true,adjacency_true,baseline_true

In [ ]:
def generate_sample_sparse(p,T,deg):
    gamma_true= generate_gamma_sparse(p,deg)
    data_true,baseline_true,adjacency_true = generate_data_uniform(end_time=T,decays=decays,gamma = gamma_true)
    return data_true,gamma_true,adjacency_true,baseline_true

In [ ]:
def generate_sample_multi(p,n_sim,T):
    gamma_true= generate_gamma(p)
    data_true,baseline_true,adjacency_true = generate_data_uniform_multi(end_time=T,decays=decays,gamma = gamma_true, n_sim=n_sim)
    return data_true,gamma_true,adjacency_true,baseline_true

In [ ]:
def generate_sample_multi_sparse(p,n_sim,T,deg):
    gamma_true= generate_gamma_sparse(p,deg)
    data_true,baseline_true,adjacency_true = generate_data_uniform_multi(end_time=T,decays=decays,gamma = gamma_true, n_sim=n_sim)
    return data_true,gamma_true,adjacency_true,baseline_true

In [ ]:
def run_prep(p,N,n_sim,T):
    data_list = []
    adj_list = []
    bas_list = []
    gamma_list = []
    theta_list = []
    c = 0
    while(c < N):
        print("data")
        try:
            data,gamma,adjacency,baseline = generate_sample_multi(p,n_sim,T)
        except Exception:
            print("Oops! large spr")
            continue
        theta = np.concatenate([baseline,adjacency.flatten()])
        print("fit")
        for d in data:
            try:
                adj,bas = est_eval(d)
            except Exception:
                print("Oops! can't fit")
                break
            c+=1
            print(c)
            data_list.append(d)
            adj_list.append(adj)
            bas_list.append(bas)
            gamma_list.append(gamma)
            theta_list.append(theta)
            if (c == N):
                break
                
    prep = [data_list,adj_list,bas_list,gamma_list,theta_list]
                
    COMP_list = []
    for i in range(p):
        COMP_list.append([])
        for s in range(2**(p-1)):
            r = np.zeros(p)
            k = s
            for j in range(p):
                if (p-j-1==i):
                    continue
                r[p-j-1] = k%2
                k = k//2
            r[i] = 1
            r = r==1
            #print(r*1)
            gamma = np.diag(np.ones(p)) == 1
            gamma[i] = r
            C = COMP(gamma,prep)
            COMP_list[i].append(C)
            
        
    with open('COMP_'+str(p)+'_'+str(T)+'.pkl', 'wb') as output:
        pickle.dump(COMP_list, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
def run_prep_sparse(p,N,n_sim,T,deg):
    data_list = []
    adj_list = []
    bas_list = []
    gamma_list = []
    theta_list = []
    c = 0
    while(c < N):
        print("data")
        try:
            data,gamma,adjacency,baseline = generate_sample_multi_sparse(p,n_sim,T,deg)
        except Exception:
            print("Oops! large spr")
            continue
        theta = np.concatenate([baseline,adjacency.flatten()])
        print("fit")
        for d in data:
            try:
                adj,bas = est_eval_sparse(d,deg)
            except Exception:
                print("Oops! can't fit")
                break
            c+=1
            print(c)
            data_list.append(d)
            adj_list.append(adj)
            bas_list.append(bas)
            gamma_list.append(gamma)
            theta_list.append(theta)
            if (c == N):
                break
                
    prep = [data_list,adj_list,bas_list,gamma_list,theta_list]
    
    tot = []
    for u in range(0,deg+1):
        for w in itertools.combinations(range(p),u):
            tot.append(w)
                
    COMP_list = []
    for i in range(p):
        COMP_list.append(dict())
        for w in tot:
            s = 0
            for c in w:
                s += 2**c
            r = np.zeros(p)
            k = s
            for j in range(p):
                if (p-j-1==i):
                    continue
                r[p-j-1] = k%2
                k = k//2
            r[i] = 1
            r = r==1
            #print(r*1)
            gamma = np.diag(np.ones(p)) == 1
            gamma[i] = r
            C = COMP(gamma,prep)
            COMP_list[i][s] = C
            
        
    with open('COMP_'+str(p)+'_'+str(T)+'_'+'sparse'+'_'+str(deg)+'.pkl', 'wb') as output:
        pickle.dump(COMP_list, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
def load_COMP(p,T):
    with open('COMP_'+str(p)+'_'+str(T)+'.pkl', 'rb') as input:
        li = pickle.load(input)
    return li

In [ ]:
def load_COMP_sparse(p,T,deg):
    with open('COMP_'+str(p)+'_'+str(T)+'_'+'sparse'+'_'+str(deg)+'.pkl', 'rb') as input:
        li = pickle.load(input)
    return li

In [ ]:
def generate_dataset(p,T,size):
    data_true_list = []
    gamma_true_list = []
    adjacency_true_list = []
    baseline_true_list = []
    c = 0
    while(c<size):
        #print("data")
        try:
            data_true,gamma_true,adjacency_true,baseline_true = generate_sample(p,T)
        except Exception:
            print("Oops! large spr")
            continue
        #print("fit")
        try:
            adj_true,bas_true = est_eval(data=data_true)
        except Exception:
            print("Oops! can't fit")
            continue
        c+=1
        print(c)
        data_true_list.append(data_true)
        gamma_true_list.append(gamma_true)
        adjacency_true_list.append(adjacency_true)
        baseline_true_list.append(baseline_true)
        
    with open('dataset_'+str(p)+'_'+str(T)+'.pkl', 'wb') as output:
        pickle.dump([data_true_list,gamma_true_list,adjacency_true_list,baseline_true_list,T], output, pickle.HIGHEST_PROTOCOL)

In [ ]:
def generate_dataset_sparse(p,T,size,deg):
    data_true_list = []
    gamma_true_list = []
    adjacency_true_list = []
    baseline_true_list = []
    c = 0
    while(c<size):
        #print("data")
        try:
            data_true,gamma_true,adjacency_true,baseline_true = generate_sample_sparse(p,T,deg)
        except Exception:
            print("Oops! large spr")
            continue
        #print("fit")
        try:
            #adj_true,bas_true = est_eval_sparse(data_true,deg)
            a = 1
        except Exception:
            print("Oops! can't fit")
            continue
        c+=1
        print(c)
        data_true_list.append(data_true)
        gamma_true_list.append(gamma_true)
        adjacency_true_list.append(adjacency_true)
        baseline_true_list.append(baseline_true)
        
    with open('dataset_'+str(p)+'_'+str(T)+'_'+'sparse'+'_'+str(deg)+'.pkl', 'wb') as output:
        pickle.dump([data_true_list,gamma_true_list,adjacency_true_list,baseline_true_list,T], output, pickle.HIGHEST_PROTOCOL)

In [ ]:
def load_dataset(p,T):
    with open('dataset_'+str(p)+'_'+str(T)+'.pkl', 'rb') as input:
        li = pickle.load(input)
    return li

In [ ]:
def load_dataset_sparse(p,T,deg):
    with open('dataset_'+str(p)+'_'+str(T)+'_'+'sparse'+'_'+str(deg)+'.pkl', 'rb') as input:
        li = pickle.load(input)
    return li

In [ ]:
def precision(gamma_true,gamma_star):
    p = len(gamma_true)
    g_true = gamma_true
    g_star = gamma_star
    return np.count_nonzero((g_true*1) * (g_star*1))/np.count_nonzero(g_star*1)

In [ ]:
def recall(gamma_true,gamma_star):
    p = len(gamma_true)
    g_true = gamma_true
    g_star = gamma_star
    return np.count_nonzero((g_true*1) * (g_star*1))/np.count_nonzero(g_true*1)

In [ ]:
def F1_score(gamma_true,gamma_star):
    p = precision(gamma_true,gamma_star)
    r = recall(gamma_true,gamma_star)
    return 2*p*r/(p+r)

In [ ]:
def evaluate(gamma_true_list,gamma_star_list,metric=F1_score):
    size = len(gamma_star_list)
    res = []
    for i in range(size):
        sc = metric(gamma_true_list[i],gamma_star_list[i])
        res.append(sc)
    return res

In [ ]:
def run_method(dataset,method,hyperparams,metric=F1_score,if_print=False):
    data_list,gamma_list,adjacency_list,baseline_list,T = dataset
    size = len(data_list)
    gamma_hat_list = []
    for i in range(size):
        gamma_hat = method(data_list[i],T,hyperparams)
        gamma_hat_list.append(gamma_hat)
        sc = metric(gamma_list[i],gamma_hat)
        if (if_print):
            print("TEST " + str(i+1) + " : " + str(sc))
    return gamma_hat_list

In [ ]:
def save_results(results):
    with open('results.pkl', 'wb') as output:
        pickle.dump(results, output, pickle.HIGHEST_PROTOCOL)

def load_results():
    with open('results.pkl', 'rb') as input:
        li = pickle.load(input)
    return li

# Methods

In [ ]:
def MDLH(data,T,hyperparams=None):
    p = len(data)
    adj, bas = est_eval(data)
    L = ModelHawkesExpKernLogLik(decay=decays,n_threads=0)
    L.fit(data);
    COMP_list = load_COMP(p,T)
    gamma_hat = np.diag(np.ones(p)) == 1
    for i in range(p):
        score = 1e100
        r_hat = []
        for s in range(2**(p-1)):
            r = np.zeros(p)
            k = s
            for j in range(p):
                if (p-j-1==i):
                    continue
                r[p-j-1] = k%2
                k = k//2
            r[i] = 1
            r = r==1
            #print(r*1)
            gamma = np.diag(np.ones(p)) == 1
            gamma[i] = r
            theta = theta_hat_eval(adj=adj,bas=bas,gamma=gamma)
            NML = L.loss(theta) + COMP_list[i][s]
            if (NML<score):
                score = NML
                r_hat = r
        gamma_hat[i] = r_hat
    return gamma_hat

In [ ]:
def MDLH_sparse(data,T,hyperparams):
    deg = hyperparams
    p = len(data)
    adj, bas = est_eval_sparse(data,deg)
    L = ModelHawkesExpKernLogLik(decay=decays,n_threads=0)
    L.fit(data);
    COMP_list = load_COMP_sparse(p,T,deg)
    gamma_hat = np.diag(np.ones(p)) == 1
    
    tot = []
    for u in range(0,deg+1):
        for w in itertools.combinations(range(p),u):
            tot.append(w)
    
    for i in range(p):
        score = 1e100
        r_hat = []
        for w in tot:
            s = 0
            for c in w:
                s += 2**c
            r = np.zeros(p)
            k = s
            for j in range(p):
                if (p-j-1==i):
                    continue
                r[p-j-1] = k%2
                k = k//2
            r[i] = 1
            r = r==1
            #print(r*1)
            gamma = np.diag(np.ones(p)) == 1
            gamma[i] = r
            theta = theta_hat_eval(adj=adj,bas=bas,gamma=gamma)
            NML = L.loss(theta) + COMP_list[i][s]
            if (NML<score):
                score = NML
                r_hat = r
        gamma_hat[i] = r_hat
    return gamma_hat

In [ ]:
def LS(data,T,hyperparams):
    p = len(data)
    penalty,C = hyperparams
    ls = HawkesExpKern(decays,gofit = 'least-squares', step = 10, max_iter=10000, tol = 1e-10, solver = 'gd', C=C)
    ls.fit(data)
    gamma_hat = ls.adjacency > TH
    return gamma_hat

In [ ]:
def MLE(data,T,hyperparams):
    p = len(data)
    penalty,C = hyperparams
    ls = HawkesExpKern(decays,gofit = 'least-squares', step = 10, max_iter=10000, tol = 1e-5, solver = 'gd')
    ls.fit(data)
    est = HawkesExpKern(decays, C = C, penalty=penalty, gofit = 'likelihood', step = 10, max_iter=10000, tol = 1e-5, solver = 'gd')
    est.fit(events=data,start = ls.coeffs+1e-5)
    gamma_hat = est.adjacency > TH
    return gamma_hat

In [ ]:
def IC_methods(data,T,hyperparams):
    p = len(data)
    adj, bas = est_eval(data)
    L = ModelHawkesExpKernLogLik(decay=decays,n_threads=0)
    L.fit(data);
    ic = hyperparams
    n = 0
    for i in range(p):
        n+= len(data[i])
    gamma_hat = np.diag(np.ones(p)) == 1
    for i in range(p):
        best = 1e100
        r_hat = []
        for s in range(2**(p-1)):
            r = np.zeros(p)
            k = s
            for j in range(p):
                if (p-j-1==i):
                    continue
                r[p-j-1] = k%2
                k = k//2
            r[i] = 1
            r = r==1
            #print(r*1)
            gamma = np.diag(np.ones(p)) == 1
            gamma[i] = r
            theta = theta_hat_eval(adj=adj,bas=bas,gamma=gamma)
            ic_penalty = 0
            k = (np.sum(r)+p-1 + p)
            if (ic == 'AIC'):
                ic_penalty = 2*k
            if (ic == 'BIC'):
                ic_penalty = k*np.log(n)
            if (ic == 'HQ'):
                ic_penalty = 2*k*np.log(np.log(n))
            #print(str(ic_penalty) +  " " + str(2*L.loss(theta)))
            score = 2*L.loss(theta) + ic_penalty
            if (score<best):
                best = score
                r_hat = r
        gamma_hat[i] = r_hat
    if (np.sum(gamma_hat*1 - np.diag(np.ones(p))) !=0):
        print("asdfa")
    #else:
    #    print(":((")
    return gamma_hat

In [ ]:
def IC_methods_sparse(data,T,hyperparams):
    p = len(data)
    L = ModelHawkesExpKernLogLik(decay=decays,n_threads=0)
    L.fit(data);
    ic,deg = hyperparams
    print("a")
    adj, bas = est_eval_sparse(data,deg)
    print("b")
    n = 0
    for i in range(p):
        n+= len(data[i])
        
    gamma_hat = np.diag(np.ones(p)) == 1
    
    tot = []
    for u in range(0,deg+1):
        for w in itertools.combinations(range(p),u):
            tot.append(w)
            
    for i in range(p):
        best = 1e100
        r_hat = []
        for w in tot:
            s = 0
            for c in w:
                s += 2**c
            r = np.zeros(p)
            k = s
            for j in range(p):
                if (p-j-1==i):
                    continue
                r[p-j-1] = k%2
                k = k//2
            r[i] = 1
            r = r==1
            #print(r*1)
            gamma = np.diag(np.ones(p)) == 1
            gamma[i] = r
            theta = theta_hat_eval(adj=adj,bas=bas,gamma=gamma)
            ic_penalty = 0
            k = (np.sum(r)+p-1 + p)
            if (ic == 'AIC'):
                ic_penalty = 2*k
            if (ic == 'BIC'):
                ic_penalty = k*np.log(n)
            if (ic == 'HQ'):
                ic_penalty = 2*k*np.log(np.log(n))
            #print(str(ic_penalty) +  " " + str(2*L.loss(theta)))
            score = 2*L.loss(theta) + ic_penalty
            if (score<best):
                best = score
                r_hat = r
        gamma_hat[i] = r_hat
    if (np.sum(gamma_hat*1 - np.diag(np.ones(p))) !=0):
        print("asdfa")
    #else:
    #    print(":((")
    return gamma_hat

In [ ]:
def ADM4(data,T,hyperparams):
    C, lasso_nuclear_ratio = hyperparams
    learner = HawkesADM4(decay=decays,C=C,lasso_nuclear_ratio=lasso_nuclear_ratio)
    learner.fit(data)
    #print((learner.adjacency>TH)*1)
    gamma_hat = (learner.adjacency>TH)
    return gamma_hat

In [ ]:
def NPHC(data,T,hyperparams):
    C, support, penalty = hyperparams
    nphc = HawkesCumulantMatching(integration_support = support, C=C, penalty = penalty, tol=1e-10, cs_ratio=.15)
    nphc.fit(data)
    gamma_hat = (nphc.adjacency>TH)
    return gamma_hat

# Synthetic Experiments

In [ ]:
uni_adj_lb = 0.1
uni_adj_ub = 0.2
uni_bas_lb = 0.5
uni_bas_ub = 1.0
bern_param = 0.3
decays = 1
TH = 0.01
p = 7
T = 700
N = 1000
n_sim = 10
size = 100
deg = 1
metric = F1_score

In [ ]:
results = dict()
save_results(results)
#generate_dataset_sparse(p,T,size,deg)
generate_dataset(p,T,size)
#dataset = load_dataset_sparse(p,T,deg)
dataset = load_dataset(p,T)
gamma_true_list = dataset[1]

### MLE

In [ ]:
penalty_list = ['l1','none','l2','elasticnet']
C_list = [500,1000,2000,5000,10000,20000,50000,100000]

In [ ]:
results = load_results()
bestest = 0
for penalty in penalty_list:
    acc_best = 0
    for C in C_list:
        if (penalty == 'none'):
            C = None
        hyperparams = [penalty,C]
        gamma_hat_list = run_method(dataset,MLE,hyperparams,metric,True)
        res = evaluate(gamma_true_list,gamma_hat_list,metric)
        acc = np.mean(res)
        if (acc > acc_best):
            print(acc)
            acc_best = acc
        if (penalty == 'none'):
            break
    print(penalty + " " + str(acc_best))
    if (bestest < acc_best):
        bestest = acc_best
        results['likelihood'+'_'+str(p)+'_'+str(T)] = bestest

save_results(results)

In [ ]:
results

### LS

In [ ]:
penalty_list = ['l1','l2','elasticnet','none']#,'nuclear']
C_list = [1,2,5,10,20,50,100,200,500,1000,2000,5000,10000,20000,50000,100000]

In [ ]:
results = load_results()

bestest = 0
for penalty in penalty_list:
    acc_best = 0
    for C in C_list:
        if (penalty == 'none'):
            C = None
        hyperparams = [penalty,C]
        gamma_hat_list = run_method(dataset,LS,hyperparams,metric,True)
        res = evaluate(gamma_true_list,gamma_hat_list,metric)
        acc = np.mean(res)
        if (acc > acc_best):
            print(acc)
            acc_best = acc
        if (penalty == 'none'):
            break
    print(penalty + " " + str(acc_best))
    if (bestest < acc_best):
        bestest = acc_best
        results['least-squares'+'_'+str(p)+'_'+str(T)] = bestest

save_results(results)

In [ ]:
results

### ADM4

In [ ]:
C_list = [1,2,5,10,20,50,100,200,500,1000,2000,5000,10000,20000,50000,100000]
ratio_list = [0,0.1,0.5,0.9,1]

In [ ]:
results = load_results()

best = 0
for ratio in ratio_list:
    for C in C_list:
        hyperparams = [C,ratio]
        gamma_hat_list = run_method(dataset,ADM4,hyperparams,metric,True)
        res = evaluate(gamma_true_list,gamma_hat_list,metric)
        acc = np.mean(res)
        if (acc > best):
            print(acc)
            best = acc
        print(str(C) + ' '  + str(ratio) + " " + str(acc))

results['ADM4'+'_'+str(p)+'_'+str(T)] = best

save_results(results)

In [ ]:
results

### NPHC

In [ ]:
C_list = [50,100,200,500,1000,2000,5000,10000,20000,50000,100000]
penalty_list = ['l1','elasticnet','none', 'l2']

In [ ]:
results = load_results()
best= 0
for penalty in penalty_list:
    acc_best = 0
    for C in C_list:
        if (penalty == 'none'):
            C = None
        hyperparams = [C,5,penalty]
        gamma_hat_list = run_method(dataset,NPHC,hyperparams,metric)
        res = evaluate(gamma_true_list,gamma_hat_list)
        acc = np.mean(res)
        if (acc > best):
            print(acc)
            best = acc
            results['NPHC'+'_'+str(p)+'_'+str(T)] = best
        print(penalty + " " + str(C) + " " + str(best))
        if (penalty == 'none'):
          break
        

save_results(results)

In [ ]:
results

### MDLH

In [ ]:
run_prep(p,N,n_sim,T)

In [ ]:
results = load_results()
gamma_hat_list = run_method(dataset,MDLH,None,metric, True)
res = evaluate(gamma_true_list,gamma_hat_list,metric)
acc = np.mean(res)
results['MDLH'+'_'+str(p)+'_'+str(T)] = acc
save_results(results)

In [ ]:
res = evaluate(gamma_true_list,gamma_hat_list,metric)
acc = np.mean(res)
acc

In [ ]:
results

### MDLH sparse

In [ ]:
run_prep_sparse(p,N,n_sim,T,deg)

In [ ]:
results = load_results()
gamma_hat_list = run_method(dataset,MDLH_sparse,deg,metric,True)

In [ ]:
res = evaluate(gamma_true_list,gamma_hat_list,metric)
acc = np.mean(res)
results['MDLH_sparse'+'_'+str(p)+'_'+str(T)] = acc
print(acc)
save_results(results)

In [ ]:
results

# IC Methods

Here we compare the log-likelihood value for the most restricted model (empty graph, only baseline vector), and the least restricted model (full graph, ML). The difference in short data is so tiny that no information criterion allow for any model other than empty graph. The difference in likelihood is of order 0.01 while it should be at least 1 so that IC allows for increasing the number of parameters.

In [ ]:
data = dataset[0][0]

In [ ]:
hyperparams = ['AIC', deg]

In [ ]:
L = ModelHawkesExpKernLogLik(decay=decays,n_threads=0)
L.fit(data);

In [ ]:
gamma_ML=MLE(data,T,['none',None])

In [ ]:
ml = estimate(data,2**p-1)

In [ ]:
v = []
for i in range(p):
    v.append(len(data[i])/T)
for i in range(p*p):
    v.append(0)
v = np.array(v)
L.loss(v)

In [ ]:
L.loss(ml.coeffs)

In [ ]:
L.loss(v) - L.loss(ml.coeffs)

##### Real-data Experiments

In [ ]:
raw = pd.read_csv("ddly-data.csv",delimiter=";")
raw = raw.drop(['dd/mm/yy'],axis = 1)
bonds = [ 'CAN_b', 'US_b', 'UK_b', 'GER_b', 'FRA_b', 'ITA_b', 'JPN_b']

In [ ]:
raw = raw.filter(bonds)

In [ ]:
def is_shock(a,i,k):
    n = len(a)
    if (i<k):
        k = i
    thresh = np.sort(a[(i-k):i+1])[::-1][int(k/5)]
    return a[i]>thresh

In [ ]:
window_size = 250
def get_events(data):
    n = len(data)
    nodes = list(data.columns)
    events = []
    for x in nodes:
        print(x)
        event = [False for i in range(n)]
        for i in range(n):
            event[i] = is_shock(data[x],i,window_size)
        tick = np.where(event)[0].tolist()
        events.append(np.asarray(tick).astype(np.double))
    return events

In [ ]:
nodes = list(raw.columns)
p = len(nodes)
T = len(raw)
data = get_events(raw).copy()

In [ ]:
for i in range(p):
    print(len(data[i]))

In [ ]:
for i in range(p):
    print(len(dataset[0][0][i]))

In [ ]:
for i in range(p):
    data[i] = 400 * data[i]/T

In [ ]:
adj,bas = est_eval(data)

In [ ]:
gamma_star = MDLH(data,400,None)

In [ ]:
gamma_star*1

In [ ]:
# 1 std, 120 w
for i in range(p):
    for j in range(p):
        if (gamma_star[i][j] and i!=j):
            print(bonds[j] + " to " + bonds[i])

In [ ]:
gamma_mle = MLE(data,400,['l1',80])*1

In [ ]:
for i in range(p):
    for j in range(p):
        if (gamma_mle[i][j] and i!=j):
            print(bonds[j] + " to " + bonds[i])